In [2]:
import json, pathlib, traceback
import numpy as np
from collections import defaultdict
from scipy.spatial.transform import Rotation as R

NEW_COMPLAINT_TYPES = {
    "category",
    "appearance",
    "handle",
    "collision",
    "joint",
    "metalink",
    "unknown",
    "connected",
}

OLD_COMPLAINT_STARTS = {
    "APPEARANCE": "appearance",
    "ABILITIES": "metalink",
    "ARTICULATION": "joint",
    "CATEGORY": "category",
    "CONNECTED": "connectedness",
    "CENTEROFMASS": "centerofmass",
    "Check if the object looks like it absolutely MUST be a soft body.": "softbody",
    "Confirm articulation:": "joint",
    "Confirm object meta links listed below:": "metalink",
    "Confirm object meta links:": "metalink",
    "Confirm object synset assignment.": "category",
    "Confirm object visual appearance.": "appearance",
    "Confirm reasonable bounding box size": "scale",
    "FILLABLE": "metalink",
    "GLASS": "appearance",
    "MIRROR": "appearance",
    "SLICER": "metalink", 
    "STRUCTURE-GLASSNESS": "appearance",
    "STRUCTURE-MULTIPLE-PIECES": "structureconnectedness",
    "STRUCTURE-THICKNESS": "structurethickness",
    "STRUCTURE-UNCLOSED": "structureunclosed",
    "STRUCTURE-TRIANGULATION": "structuretriangulation",
    "STRUCTURE-APPEARANCE": "appearance",
    "STRUCTURE-SYNSET": "category",
    "SUBSTANCE": "substanceness",
    "SYNSET": "category",
    "Was at least one of the collision mesh candidates acceptable?" : "collision",
    "Confirm object properties:": "property", 
}

complaints_by_model = defaultdict(list)
scales_by_model = {}
orientations_by_model = {}
complaints_by_type = defaultdict(list)
for path in pathlib.Path(r"D:\ig_pipeline").glob("qa-2025/*/*/*.json"):
    try:
        batch = path.parts[-3]
        mdl = path.stem
        object_json = json.loads(path.read_text())

        # Process scale and orientation
        scale = np.array(object_json["scale"])
        if not np.allclose(scale, 1, atol=1e-3):
            assert mdl not in scales_by_model, f"Duplicate scale for {mdl} from {batch}"
            scales_by_model[mdl] = scale.tolist()
        
        orientation = np.array(object_json["orientation"])
        rot = R.from_quat(orientation)
        if rot.magnitude() > np.deg2rad(2):
            assert mdl not in orientations_by_model, f"Duplicate orientation for {mdl} from {batch}"
            orientations_by_model[mdl] = orientation.tolist()

        complaints = object_json["complaints"]

        # Update all of the QC: complaints to match the author's batch
        for c in complaints:
            if c["message"].startswith("QC: "):
                c["type"] = c["message"][4:]
                assert c["type"] in NEW_COMPLAINT_TYPES
                c["additional_info"] = ""
                c["complaint"] = f"quick complaint added during qa-2025 batch {batch}"
                del c["message"]
            else:
                for prefix_candidate, type_candidate in OLD_COMPLAINT_STARTS.items():
                    if c["message"].startswith(prefix_candidate):
                        c["type"] = type_candidate
                        c["additional_info"] = c["message"] # [len(prefix_candidate):]
                        del c["message"]
                        break
                else:
                    raise ValueError(f"Unrecognized complaint: {c['message']}")

        complaints_by_model[mdl].extend(complaints)

        for c in complaints:
            complaints_by_type[c["type"]].append(c)
    except:
        print(f"Error processing {path}")
        print(traceback.format_exc())

In [3]:
with open(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json", "r") as f:
    providers = {x.split("-")[1]: y for x, y in json.load(f)["providers"].items()}

In [4]:
import collections

complaints_by_target = collections.defaultdict(list)
for obj, complaints in complaints_by_model.items():
    if obj not in providers:
        print(f"Skipping {obj} because it is not in the object inventory")
        continue
    complaints_by_target[providers[obj]].extend(complaints)

In [5]:
complaints_by_target

defaultdict(list,
            {'objects/task_relevant-xy': [{'object': 'bag_of_rubbish-qfkzkd',
               'complaint': 'quick complaint added during qa-2025 batch qa-2025-eric',
               'processed': False,
               'new': True,
               'type': 'appearance',
               'additional_info': ''},
              {'object': 'belt-gqwddb',
               'complaint': 'white (should be grey)',
               'processed': True,
               'new': True,
               'type': 'appearance',
               'additional_info': 'APPEARANCE: Confirm object visual appearance.\nRequirements:\n- make sure the object has a valid texture or appearance (e.g., texture not black,\n       transparency rendered correctly, etc).\n- make sure any glass parts are transparent (would this object contain glass? e.g.\n       wall pictures, clocks, etc. - anything wrong)\n- compare the object against the 3ds Max image that should open up now.'},
              {'object': 'bok_choy-jpkewd',


In [6]:
# Save all the complaints
import os
for target, complaints in complaints_by_target.items():
    with open(os.path.join(r"D:\ig_pipeline\cad", *(target.split("/")), "complaints.json"), "w") as f:
        json.dump(complaints, f, indent=4)

In [7]:
# Assert that all the target files contain only type-annotated complaints
for path in pathlib.Path(r"D:\ig_pipeline").glob("cad/*/*/complaints.json"):
    complaints = json.loads(path.read_text())
    for c in complaints:
        if not "type" in c: print(f"Missing type in {path}: {c}")
        if not "additional_info" in c: print(f"Missing additional_info in {path}: {c}")
        if not "complaint" in c: print(f"Missing complaint in {path}: {c}")
        if not "processed" in c: print(f"Missing processed in {path}: {c}")

In [8]:
# Save all the orientation and scale edits
with open(r"D:\ig_pipeline\metadata\orientation_and_scale_edits.json", "w") as f:
    json.dump({
        "scales": scales_by_model,
        "orientations": orientations_by_model,
    }, f, indent=4)

In [9]:
# What scene objects are scaled?
for scaled_model, scale in scales_by_model.items():
    if providers[scaled_model].startswith("scenes"):
        print(scaled_model, scale)

bzmdxc [10.0, 10.0, 10.0]
bwteqh [1.2100000381469727, 1.2100000381469727, 1.2100000381469727]
dfgurb [0.683013379573822, 0.683013379573822, 0.683013379573822]
hwrflj [1.100000023841858, 1.100000023841858, 1.100000023841858]
obixxh [1.2100000381469727, 1.2100000381469727, 1.2100000381469727]
qrqzvs [0.683013379573822, 0.683013379573822, 0.683013379573822]
rizrsp [1.9487173557281494, 1.9487173557281494, 1.9487173557281494]
ymhxqk [1.2100000381469727, 1.2100000381469727, 1.2100000381469727]
yyuiva [0.9090908765792847, 0.9090908765792847, 0.9090908765792847]
zlxfxt [1.2100000381469727, 1.2100000381469727, 1.2100000381469727]
lyigsj [1.4641001224517822, 1.4641001224517822, 1.4641001224517822]
utgixp [2.1435890197753906, 2.1435890197753906, 2.1435890197753906]
pnrdxh [0.9090908765792847, 0.9090908765792847, 0.9090908765792847]
mcjlhs [1.3310000896453857, 1.3310000896453857, 1.3310000896453857]
eeaimz [0.683013379573822, 0.683013379573822, 0.683013379573822]
jtjcrx [0.62092125415802, 0.620921